In [ ]:
"""
一、Module与Sequential
    1.介绍
    Module是torch中用于定义一个神经网络模型的基类，通过继承该类可以定义自己需要的模型或者Block。在Module定义自己的模型时，模型参数
可以在__init__方法中定义，此外要注意在重写该方法时要调用一次父类的该方法。模型前向计算的过程由forward方法定义，模型前向计算的过程是
由__call__方法实现的，可直接将模型的实例当作函数，如y = model(x)得到输出y。
    Sequential用于辅助定义模型，它将输入的层或者Block堆叠在一起生成一个Block，前一层的输出作为下一层的输入。

    2.模型定义--示例程序
    import torch
    from torch import nn
    from torch.utils.data import Dataset, DataLoader
    
    class Classifier(nn.Module):
        def __init__(self):
            super(Classifier, self).__init__()      #调用父类方法，固定格式
            
            self.prenet = nn.Linear(40, 256)
            self.hidden = nn.Sequential(
                nn.Linear(256, 128),
                nn.Relu(),
                nn.Linear(128, 128),
                nn.Relu()
                )
            self.predict = nn.Linear(128, 29)
        
        def forward(self, x):
            out = self.prenet(x)
            
            out = self.hidden(out)
            
            out = self.predict(out)
            
            return out
"""

In [ ]:
"""
二、模型训练、保存与加载
    1.介绍
    模型训练不仅要有预定义的模型，还要有相应的损失函数、优化方法。torch中2个最典型的损失函数为MSELoss、CrossEntropy，他们都支持__call__
方法，在计算模型预测值和真实值时直接对损失函数的实例进行函数式调用即可计算损失。torch为常见优化算法定义了相关类型，在具体操作时使用
.zero_grad()、.step()即可完成参数更新。
    在具体训练时每次计算包含前向计算与反向传播2个环节，前者由__call__方法实现，后者由torch支持的自动求导实现
    
    2.设备选取
    模型训练时可选择在cpu、gpu上运行，相应的模型以及要计算的数据必须在同一设备中。
    
    device = "cuda" if torch.cuda.is_available() else "cpu"        #选取可用设备，假定为gpu
    
    model = Classifier()
    model.to(device)                 #将模型加入gpu
    model.cpu()                      #将模型加入cpu
    
    3.模型训练--示例程序
    #假设程序将数据集分为训练集与验证集，在训练集上训练模型，在验证集上评价模型
    #dataset表示一个分类问题训练集
    #val_set表示一个分类问题的验证集
    import torch
    from torch import nn
    from torch import Dataset, DataLoader
    
    BATCH_SIZE = 32
    tr_loader = DataLoader(
        dataset,
        batch_size = BATCH_SIZE,
        shuffle = True
    )
    val_loader = DataLoader(
        val_set,
        batch_size = BATCH_SZIE,
        shuffle = False
        )
        
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    #定义模型、评价准则、优化方法
    model = Classifier()
    model.to(device)
    criterion = nn.CrossEntropy()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)      #将模型参数交给优化器
    
    #设置早停,不是必要的
    max_iter = 100
    early_stop = 0
    max_val_loss = torch.inf
    
    n_epochs = 2000
    for i in range(n_epochs):
        for data, labels in tr_loader:
            #将数据加入gpu
            data, labels = data.to(device), labels.to(device)
            
            #前向计算
            y = model(data)
            loss = criterion(y, labels)
            y_pred = torch.argmax(loss, dim=1)
            
            #反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        val_loss = 0
        for k, batch in enumerate(val_loader):
            #禁用梯度，加快计算速度
            with torch.no_grad():
                data, labels = data.to(device), labels.to(device)
                
                y = model(data)
                loss = criterion(y, labels)
                val_loss = val_loss + loss.cpu().item()    #将0维张量转为浮点数
                
            return val_loss/(k + 1)
        
        if val_loss < max_val_loss:
            max_val_loss = val_loss
            early_stop = 0
            torch.save(model.state_dict(), "model.ckpt")    #保存较好的模型
        else:
            early_stop = early_stop + 1
            
        if early_stop >= max_iter:
            break
            
    4.保存与加载模型
        torch.save(model.state_dict(), "model.ckpt")
        
        model = Classifier()
        model.load_state_dict(torch.load("model.ckpt"))
                
    
"""